In [1]:
import pandas as pd
import json
from tqdm.notebook import tqdm

In [2]:
def get_like_data(path):
    df = pd.read_csv(path, sep='\t', header=None)
    df[2] = df[2].apply(lambda x: x.replace('-1 {"value":"love"} ', ''))
    df['user'] = df[2].apply(lambda x: json.loads(x)['subjects'][0]['id'])
    df['track'] = df[2].apply(lambda x: json.loads(x)['objects'][0]['id'])
    df = df[['user', 'track']]
    df.drop_duplicates(inplace=True)
    df.to_pickle('likes.pkl')
    
    return df

In [3]:
get_like_data('ThirtyMusic/relations/love.idomaar')

,user,track
0,44542,2785601
1,44542,2785590
2,44542,143076
3,44542,143037
4,44542,143052
...,...,...
4106334,39896,3208469
4106335,39896,2756716
4106336,39896,3644495
4106338,39896,2247239


In [19]:
likes_df = pd.read_pickle('likes.pkl')
sessions_df = pd.read_pickle('sessions_counts.pkl')
tracks_df = pd.read_pickle('tracks_df.pkl')
tracks_df.rename(columns={'track_id':'track', 'artist_id':'artist'}, inplace=True)

In [20]:
likes_df_artists = likes_df.merge(tracks_df, on='track', how='inner')

In [21]:
likes_df_artists

,user,track,artist
0,44542,2785601,349295
1,44542,2785590,349295
2,44542,143076,18754
3,13516,143076,18754
4,44542,143037,18754
...,...,...,...
2451033,39880,2254906,281944
2451034,39880,2254906,281944
2451035,39880,263514,32971
2451036,39880,3194439,395571


In [26]:
def like_advantage(user_id, artist_counts):
    for k, v in artist_counts.items():
        if len(likes_df_artists[(likes_df_artists.user == user_id)
                                & (likes_df_artists.artist == k)]) == 1:  # we have this pair in like dfii
            sessions_df.iloc[user_id]['artist_counts'][k] *= 10

In [27]:
sessions_df = sessions_df[0:5000]
for session in tqdm(range(len(sessions_df))):
    like_advantage(session, sessions_df.iloc[session]['artist_counts'])

In [28]:
sessions_df.to_pickle('sessions_likes.pkl')